In [642]:
# imports
import pandas as pd

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# cnn imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras.layers.normalization import BatchNormalization
#from tensorflow.keras.layers.advanced_activations import LeakyReLU

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import Layer
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


#from tensorflow.keras import backend as K
#from tensorflow.keras import layers
#from tensorflow.python.keras.layers import Lambda
#from tensorflow.keras.callbacks import TensorBoard
#from tensorflow.keras.backend import sparse_categorical_crossentropy
#from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

In [643]:
# Load data
bodies_df = pd.read_csv("train_bodies.csv")
stances_df =  pd.read_csv("train_stances.csv")

In [644]:
# Examine data shape
bodies_df.shape

(1683, 2)

In [645]:
stances_df.shape

(49972, 3)

In [646]:
combined_df = pd.DataFrame(columns=['Headline', 'articleBody', 'Stance'])
combined_df.head()

,Headline,articleBody,Stance


In [647]:
for i in range (1250):
    combined_df.at[i, 'Headline'] =  stances_df.at[i,'Headline']
    combined_df.at[i,'articleBody'] = bodies_df.loc[bodies_df['Body ID'] == stances_df.at[i,'Body ID'], ['articleBody']].iloc[0]['articleBody']
    combined_df.at[i,'Stance'] = stances_df.at[i,'Stance']

In [648]:
combined_df.head()

,Headline,articleBody,Stance
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,agree
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,unrelated
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",disagree


In [649]:
combined_df.to_csv("combined_stances.csv")

In [650]:
# store column names for convenience
column_names = combined_df.columns
display(column_names)

Index(['Headline', 'articleBody', 'Stance'], dtype='object')

In [687]:
# Extract column for just labels
labels = combined_df['Stance']

In [718]:
new_labels = labels.copy()

In [719]:
new_labels[new_labels=='agree']=1
new_labels[new_labels=='unrelated']=0.25
new_labels[new_labels=='disagree']=0
new_labels[new_labels=='discuss']=0.75

In [720]:
new_labels

0       0.25
1          1
2       0.25
3       0.25
4          0
        ... 
1245    0.25
1246    0.25
1247    0.25
1248    0.75
1249    0.25
Name: Stance, Length: 1250, dtype: object

In [691]:
for i in range (1250):
    new_labels.at[i, 'Stance'] = labels.loc[bodies_df['Body ID'] == stances_df.at[i,'Body ID'], ['articleBody']].iloc[0]['articleBody']
    combined_df.at[i,'Stance'] = stances_df.at[i,'Stance']

<bound method NDFrame.head of 0       unrelated
1           agree
2       unrelated
3       unrelated
4        disagree
          ...    
1245    unrelated
1246    unrelated
1247    unrelated
1248      discuss
1249    unrelated
Name: Stance, Length: 1250, dtype: object>

In [652]:
# Drop columns that are strings
all_data_processed = combined_df.drop(['Stance'], axis=1)

In [653]:
# Set some variables to hold test, dev, and training data.
test_data, test_labels = all_data_processed[1000:], labels[1000:]
#dev_data, dev_labels = all_data_processed[17505:23340], labels[17505:23340]
train_data, train_labels = all_data_processed[:1000], labels[:1000]

In [654]:
# Used GLoVE embeddings
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

In [657]:
import gensim

In [658]:
# NLTK for NLP utils
import nltk
# Helper libraries.
#from w266_common import utils, vocabulary, tf_embed_viz
import utils, vocabulary, tf_embed_viz

In [661]:
def sent_to_ids(sentences):
    """
    converting a list of strings to a list of lists of word ids
    """
    
    # TODO - Update Regular Expression for parsing (TINA)
    # Improvement Option (Remove trailing 's') Check Missing list for optimization
    
    #re.sub('[^A-Za-z0-9]+', '', mystring)
    #cleanString = re.sub('\W+','', string ) 
    
    bad_chars = [';', ':', '!', "*"] 
    
    text_ids = []
    example = []
    sentences = sentences.replace('.','')
    sentences = sentences.replace('\'','')
    sentences = sentences.replace(',','')
    sentences = sentences.replace('$','')
    sentences = sentences.replace('/',' ')
    sentences = sentences.replace('','')
    for word in sentences.split(' '):
        #word.lower()
        temp = word.lower()
        #temp.replace('\'','')
        if temp in embeddings_dict:
            example.append(embeddings_dict[temp])
        else:
            print("Missing ", temp)            
    text_ids.append(example)

    return  text_ids   

In [662]:
all_data_emb = all_data_processed.copy()

In [663]:
for i in range (1250):
    all_data_emb.at[i, 'Headline'] =  np.array(sent_to_ids(all_data_processed.at[i,'Headline']))
    all_data_emb.at[i, 'articleBody'] =  np.array(sent_to_ids(all_data_processed.at[i,'articleBody']))

Missing  film

seth
Missing  sony’s
Missing  biopic

danny
Missing  isaacsons
Missing  years

negotiations
Missing  it’s
Missing  talks

of
Missing  stage

insiders
Missing  list

wozniak
Missing  "woz"
Missing  storm

the
Missing  (wadi
Missing  gaza)
Missing  meters

gaza
Missing  down

the
Missing  110000
Missing  summer

the
Missing  entering

gaza
Missing  al-midana
Missing  sea

evacuated
Missing  al-bureij
Missing  strip

the
Missing  (wadi
Missing  gaza)
Missing  al-nuseirat
Missing  al-moghraqa
Missing  habesor
Missing  hebron

israeli
Missing  area

locals
Missing  hazard

this
Missing  dams

in
Missing  homes

in
Missing  lifenews
Missing  opoila
Missing  vessels



-
Missing  officials

at
Missing  testicles

-
Missing  adventures

-
Missing  say

interestingly
Missing  (reuters)
Missing  wednesday

the
Missing  montreal

(reporting
Missing  hopkins;
Missing  abocar)
Missing  arachnophobes
Missing  bunburys
Missing  "spiderman"
Missing  seemed

perth
Missing  online

earlie

Missing  "pub-6608306193529351";
Missing  google_ad_width
Missing  468;
Missing  google_ad_height
Missing  60;
Missing  google_ad_format
Missing  "468x60_as";
Missing  google_ad_type
Missing  "text";
Missing  google_ad_channel
Missing  "0054321535";
Missing  google_color_border
Missing  "000000";
Missing  
Missing  
Missing  -->
Missing  extra:
Missing  back-from-the-dead
Missing  re-started
Missing  figure

father
Missing  rhythm

however
Missing  recovered

the
Missing  debunked:
Missing  robocop-style
Missing  knightscope
Missing  true

sorry
Missing  robocop-style
Missing  campus

the
Missing  knightscope
Missing  knightscope
Missing  stephens

"so
Missing  photos"
Missing  said

the
Missing  guards

knighscope
Missing  security-guard-mimicking
Missing  testers

"obviously
Missing  [a
Missing  customer]"
Missing  "but
Missing  there"

stephens
Missing  broke

"they
Missing  not"
Missing  said

"unfortunately
Missing  things"
Missing  added

so
Missing  says;
Missing  punishment

th

Missing  to?
Missing  it:
Missing  expected?
Missing  things

source:
Missing  sale

the
Missing  years

it
Missing  tee-off
Missing  planes

the
Missing  skinnpälsarna
Missing  acres

however
Missing  furnishing

sabine
Missing  rollinger
Missing  said:
Missing  "this
Missing  more

"this
Missing  mini-mansion

"also
Missing  tee-off
Missing  lawn

"here
Missing  golfing

"a
Missing  boats

"this
Missing  mälarens
Missing  stockholm

tiger
Missing  (reuters)

"this
Missing  archipelago

"the
Missing  lake

"the
Missing  occasion

"the
Missing  dining

"you
Missing  summer

"apart
Missing  skinnpälsarna
Missing  metres"

vladi
Missing  brown’s
Missing  on

a
Missing  released

the
Missing  wilson

according
Missing  shot

“a
Missing  (whose
Missing  protected)
Missing  brown”
Missing  “because
Missing  second”

the
Missing  won’t
Missing  

the
Missing  brown’s
Missing  incident

they
Missing  driver’s
Missing  wilson’s
Missing  gunshots

the
Missing  it’s
Missing  be

days
Missing  ch

Missing  report:
Missing  misfeatures
Missing  owners:
Missing  smartwatch
Missing  shower

to
Missing  "water
Missing  resistant"
Missing  smartwatches
Missing  (read:
Missing  wear)
Missing  ip67
Missing  ratings

the
Missing  smartwatches
Missing  night

will
Missing  urbane?
Missing  wonder

via:
Missing  zehaf-bibeau
Missing  guard

by
Missing  confiscated

it
Missing  zehaf-bibeau
Missing  ottawa’s
Missing  parliament

zehaf-bibeau
Missing  vickers

vickers:
Missing  ottawa

mr
Missing  zehaf-bibeau
Missing  dead

michael
Missing  zehaf-bibeau
Missing  credit:
Missing  @cbcottawa
michael
Missing  zehaf-bibeau
Missing  ottawa
canadian
Missing  “terror
Missing  attack”
Missing  actions

the
Missing  rouleau-couture

corporal
Missing  cirillo’s
Missing  “wonderful
Missing  man”
Missing  highlanders

canberra:
Missing  parliament

in
Missing  attempts

the
Missing  country

“he
Missing  world”
Missing  cirillo’s
Missing  newspaper

live-
Missing  attack

she
Missing  “he
Missing  son

Missing  3077reddit70
Missing  
Missing  32

(rns)
Missing  heaven

“paradise
Missing  god’s
Missing  creatures”
Missing  reassuringly

pope
Missing  gibson

Missing  caption

pope
Missing  gibson

Missing  morrow

it
Missing  peter’s
Missing  francis’
Missing  “people’s
Missing  pope”
Missing  times

there’s
Missing  problem:
Missing  happened

yes
Missing  dog

so
Missing  fact?

part
Missing  pope’s
Missing  “new
Missing  heaven”
Missing  “new
Missing  earth”
Missing  “the
Missing  being”

the
Missing  francis’
Missing  heaven

“one
Missing  christ”
Missing  ago

the
Missing  misleadingly:
Missing  “paradise
Missing  animals?
Missing  doesn’t
Missing  out”
Missing  wasn’t
Missing  however

the
Missing  vaticanista
Missing  further:
Missing  “the
Missing  pets:
Missing  “paradise
Missing  creatures”

vecchi
Missing  pope’s
Missing  vi’s
Missing  remark

but
Missing  francis’
Missing  story

the
Missing  “we
Missing  animals”
Missing  (that
Missing  bible)

the
Missing  “one
Missing  

...

Missing  fence-jumper
Missing  muted:

after
Missing  half-flight
Missing  family’s
Missing  80-foot-long
Missing  addresses

gonzalez
Missing  counter-assault
Missing  incident

secret
Missing  incident

sothey
Missing  lied?

on
Missing  wapos
Missing  leonnig?
Missing  neighbor?s
Missing  cats?
Missing  defendant?s
Missing  prison
source:
Missing  (isis)
Missing  kobani
Missing  grenades

the
Missing  verified

according
Missing  state

"we
Missing  [via
Missing  video]"
Missing  "analysts
Missing  it"

us
Missing  authorities

kobani
Missing  september

have
Missing  story?
Missing  confirmed
ctv
Missing  shot
police
Missing  hill

more
Missing  dalek-style
Missing  robocops
Missing  year

british
Missing  44

in
Missing  sotloff
Missing  worker

haines
Missing  august

the
Missing  “this
Missing  ordeal

“we
Missing  takes”

cameron
Missing  situation

the
Missing  “the
Missing  isil”
Missing  administration’s
Missing  “our
Missing  kingdom

“the
Missing  world”

the
Missing  sotl

Missing  light”

“her
Missing  mother’s
Missing  doesn’t
Missing  more!”

as
Missing  priest’s
Missing  “experience”
Missing  rumors

though
Missing  “holy
Missing  mother”

“i
Missing  woman…”

given
Missing  o’neal’s
Missing  priest

[image
Missing  today]
Missing  "spiderman"
Missing  
west
Missing  "superpowers"
Missing  "searing
Missing  sensation"
Missing  sternum
the
Missing  island
dylan
Missing  
dylan
Missing  skin
he
Missing  
advertisement
Missing  
Missing  
"it
Missing  feeling"
Missing  
mr
Missing  worsen

an
Missing  antihistamines
mr
Missing  hospital
"well
Missing  luckily"
Missing  
two
Missing  "had
Missing  idea"
Missing  dermatologist
the
Missing  out
mr
Missing  butcher’s
Missing  cleaver

suresh
Missing  india’s
Missing  girl’s
Missing  screams

a
Missing  him

he
Missing  butcher’s
Missing  cleaver

it
Missing  street

kumar
Missing  attack

local
Missing  said:
Missing  ‘no
Missing  crime’

more:
Missing  robbery

‘we
Missing  it’s
Missing  stopped

‘this
Mis

Missing  etats-unis:
Missing  affirme
Missing  iraq

iraq’s
Missing  said

“the
Missing  al-dulaimi
Missing  al-dulaimi
Missing  explosions”
Missing  wednesday

“the
Missing  al-dulaimi
Missing  esraa
Missing  mahel
Missing  al-qaisi
Missing  al-dulaimi”
Missing  said

the
Missing  said

security
Missing  month

they
Missing  watch

thats
Missing  expensive

grail
Missing  42mm
Missing  costs

thats
Missing  watchs
Missing  "gold-plated"
Missing  "gold-filled"
Missing  products

the
Missing  349

but
Missing  gold-framed
Missing  cartiers
Missing  jager-lecoultres
Missing  reverso
Missing  23000

even
Missing  product

grail
Missing  7999
Missing  today

the
Missing  (isis)
Missing  group

the
Missing  event

the
Missing  britain-based
Missing  cache

scroll
Missing  video

a
Missing  (isis)
Missing  kobane
Missing  syria-turkey
Missing  border

a
Missing  kobane
Missing  said

the
Missing  kobane
Missing  border

the
Missing  200000
Missing  air

on
Missing  usa

the
Missing  kobani
M

Missing  pentagon:
Missing  sorkins
Missing  wozniak:
Missing  rogen

the
Missing  isaacsons
Missing  luminary

sony
Missing  mashables
Missing  december

this
Missing  macintosh

the
Missing  relationships

"it’s
Missing  biopic;
Missing  it’s
Missing  it’s
Missing  that"
Missing  "he’s
Missing  he’s
Missing  life"

steve
Missing  script

"it
Missing  it’s
Missing  not"
Missing  "but
Missing  it’s
Missing  feeling"

wozniak
Missing  ("personalities
Missing  closer")
Missing  rogen

steve
Missing  same

ridley
Missing  "1984"
Missing  tablet

when
Missing  apple

although
Missing  "bought"
Missing  market

the
Missing  new

with
Missing  20

nearly
Missing  company

the
Missing  simple:
Missing  else

in
Missing  post-napster
Missing  margin

ten
Missing  category

after
Missing  swirl

the
Missing  motorolas
Missing  itunes

still
Missing  jobss
Missing  product

steve
Missing  jobss
Missing  keynote

all
Missing  iphone

the
Missing  jobss
Missing  history

steve
Missing  amazing

in

Missing  he’s
Missing  truckin’
Missing  isn’t
Missing  he’s
Missing  hoax

the
Missing  msmbc
Missing  “found
Missing  home”
Missing  news

with
Missing  life

the
Missing  nelson’s
Missing  “police”

“there
Missing  play”
Missing  aldeson

here’s
Missing  it’s
Missing  msmbc
Missing  imagination

there
Missing  nelson’s
Missing  “about”
Missing  “terms
Missing  use”
Missing  “privacy”
Missing  nowhere

even
Missing  don’t
Missing  isn’t

“determining
Missing  weeks”
Missing  “it’s
Missing  death”

the
Missing  “story
Missing  verified”

of
Missing  hasn’t
Missing  that’s
Missing  isn’t
Missing  true

the
Missing  isn’t
Missing  them

willie
Missing  it’s
Missing  people

[image
Missing  world]
Missing  yemen

the
Missing  33-year-olds
Missing  death

the
Missing  times

al
Missing  thwe
Missing  demands

this
Missing  congressmans
Missing  300000
Missing  bellagio’s
Missing  revealed

bloomberg
Missing  casino’s
Missing  private

on
Missing  “encouraging
Missing  sign”
Missing  bottl

In [664]:
all_data_emb

,Headline,articleBody
0,"[[[0.49725, -1.1949, 0.37137, -0.081662, 0.691...","[[[-0.31355, 0.42849, 0.35756, 0.42991, 0.2927..."
1,"[[[1.1569, -0.37447, 1.0013, -0.68139, -0.1584...","[[[1.1569, -0.37447, 1.0013, -0.68139, -0.1584..."
2,"[[[-0.014143, 0.22457, -0.76705, -1.3631, 1.35...","[[[-0.56148, -0.31749, 0.043532, -0.6, 0.54374..."
3,"[[[0.53654, 0.54846, -0.020284, 1.1767, 0.0991...","[[[-0.16768, 1.2151, 0.49515, 0.26836, -0.4585..."
4,"[[[1.5571, -0.36854, -0.37088, 0.79536, 0.6130...","[[[0.53661, -0.33872, 0.28772, -0.72584, -0.04..."
...,...,...
1245,"[[[0.086888, -0.19416, -0.24267, -0.33391, 0.5...","[[[0.19086, 0.24339, 1.2768, -0.038207, 0.6094..."
1246,"[[[0.97263, 1.0474, -0.25339, 0.50992, -0.0794...","[[[0.1011, -0.16566, 0.22035, -0.10629, 0.4692..."
1247,"[[[0.69639, 0.30378, 0.23044, -0.11817, 0.1819...","[[[0.52042, -0.8314, 0.49961, 1.2893, 0.1151, ..."
1248,"[[[1.1313, 0.65734, 0.19981, -0.62265, 0.37198...","[[[0.83907, -0.97353, 0.023372, -0.18497, 0.01..."


In [674]:
all_data_processed['Headline'][1]

'Hundreds of Palestinians flee floods in Gaza as Israel opens dams'

In [734]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(all_data_processed['Headline'])
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(all_data_processed['Headline'])
#print(encoded_docs)

In [735]:
embeddings_index_ex = dict()
with open("glove.6B.50d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index_ex[word] = coefs
#    f.close()
print('Loaded %s word vectors.' % len(embeddings_index_ex))

Loaded 400000 word vectors.


In [736]:
t.word_index.items()

dict_items([('in', 1), ('to', 2), ('of', 3), ('the', 4), ('a', 5), ('isis', 6), ('is', 7), ('on', 8), ('for', 9), ('at', 10), ('and', 11), ('after', 12), ('with', 13), ('michael', 14), ('by', 15), ('video', 16), ('says', 17), ('state', 18), ('islamic', 19), ('boko', 20), ('haram', 21), ('as', 22), ('not', 23), ('journalist', 24), ('his', 25), ('foley', 26), ('kim', 27), ('james', 28), ('was', 29), ('audio', 30), ('brown', 31), ('jong', 32), ('claims', 33), ('us', 34), ('missing', 35), ('new', 36), ('has', 37), ('american', 38), ('report', 39), ('ebola', 40), ('have', 41), ('that', 42), ('man', 43), ('apple', 44), ('be', 45), ('who', 46), ('u', 47), ('s', 48), ('leader', 49), ('steve', 50), ('it', 51), ('militants', 52), ('shooting', 53), ('woman', 54), ('kidnapped', 55), ('watch', 56), ('reports', 57), ('reportedly', 58), ('from', 59), ('ceasefire', 60), ('jobs', 61), ('shot', 62), ('hoax', 63), ('un', 64), ('girls', 65), ('over', 66), ('dead', 67), ('ottawa', 68), ('iraq', 69), ('behe

In [737]:
# create a weight matrix for words in training docs
embedding_matrix_ex = np.zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index_ex.get(word)
    if embedding_vector is not None:
        embedding_matrix_ex[i] = embedding_vector

In [738]:
vocab_size

2580

In [739]:
num_steps = 1
hidden_size = 10

model2 = Sequential()
#model2.add(Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'))
#model.add(Embedding(vocab, hidden_size, input_length=num_steps))

#model2.add(Embedding(vocab_size, hidden_size, input_length=num_steps))
model2.add(Embedding(vocab_size, 50, weights=[embedding_matrix_ex], input_length=4, trainable=False))

#model2.add(LSTM(hidden_size, return_sequences=True))
#model2.add(LSTM(hidden_size, return_sequences=True))

#model.add(Conv2D(64, (3, 3)))

model2.add(Flatten())
#model2.add(Dense(1, activation='sigmoid'))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))


#model2.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(vocab_size)))
#model2.add(Activation('softmax'))

In [740]:
# summarize the model
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 4, 50)             129000    
_________________________________________________________________
flatten_8 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_23 (Activation)   (None, 1)                 0         
Total params: 129,201
Trainable params: 201
Non-trainable params: 129,000
_________________________________________________________________
None


In [741]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(all_data_processed['Headline'])
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[368, 438, 135, 369, 13, 10, 248, 1621, 1622, 317, 145, 1114, 1115, 804, 136, 1623, 12, 368, 1624], [1116, 3, 1625, 1626, 1627, 1, 1117, 22, 1118, 1628, 1119], [79, 86, 1120, 8, 439, 3, 50, 61, 249, 58, 1121, 112, 612, 370, 9, 613], [805, 11, 44, 1, 318, 9, 1122, 1123, 44, 282, 614, 227, 1124, 1, 1629], [83, 171, 125, 1125, 250, 11, 106, 156, 25, 615], [1126, 228, 1630, 80, 1127, 172, 319, 3, 1631, 1128, 1, 1129, 173, 157, 203, 506, 251], [1632, 806, 1633, 1634, 1635, 807, 1, 1636, 76, 616, 1637, 617, 1638, 320], [618, 3, 6, 619, 1130, 22, 252, 253, 58, 507], [137, 1639, 174, 618, 1640, 7, 4, 1641, 63, 59, 1642, 321], [158, 1131, 808, 371, 809, 15, 6], [1132, 1133], [54, 229, 1, 101, 7, 23, 87, 322, 121, 69, 17], [55, 126, 116, 187, 33, 60, 102, 13, 20, 21, 42, 80, 508, 35, 65, 254], [93, 42, 283, 113, 440, 323, 441, 284, 138, 620, 810], [1134, 372, 76, 1135, 2, 1136, 23, 373, 1, 36, 509], [811, 621, 1137, 812, 442, 2, 443, 813, 374, 146, 1138, 9, 4, 622, 51, 147, 814, 81, 2, 45, 5, 6

In [742]:
model2.compile(optimizer='adam', loss='binary_crossentropy')

In [750]:
# fit the model
batch_size = 30
model2.fit(padded_docs, new_labels, batch_size=batch_size, epochs=50)


Epoch 1/50
1250/1250 [==============================] - 0s 130us/step - loss: 0.6541
Epoch 2/50
1250/1250 [==============================] - 0s 112us/step - loss: 0.6543
Epoch 3/50
1250/1250 [==============================] - 0s 100us/step - loss: 0.6543
Epoch 4/50
1250/1250 [==============================] - 0s 109us/step - loss: 0.6543
Epoch 5/50
1250/1250 [==============================] - 0s 108us/step - loss: 0.6541
Epoch 6/50
1250/1250 [==============================] - 0s 110us/step - loss: 0.6541
Epoch 7/50
1250/1250 [==============================] - 0s 107us/step - loss: 0.6544
Epoch 8/50
1250/1250 [==============================] - 0s 115us/step - loss: 0.6541
Epoch 9/50
1250/1250 [==============================] - 0s 115us/step - loss: 0.6542
Epoch 10/50
1250/1250 [==============================] - 0s 125us/step - loss: 0.6539
Epoch 11/50
1250/1250 [==============================] - 0s 102us/step - loss: 0.6543
Epoch 12/50
1250/1250 [==============================] - 0s 105

In [751]:
results = model2.predict(padded_docs)

In [752]:
# evaluate the model
#loss, accuracy =
accuracy = model2.evaluate(padded_docs, new_labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 65.200031


In [753]:
results[:20]

array([[0.38638723],
       [0.7668729 ],
       [0.44041687],
       [0.26257262],
       [0.34788358],
       [0.56804913],
       [0.23889074],
       [0.3207721 ],
       [0.390614  ],
       [0.44344983],
       [0.31266505],
       [0.38610175],
       [0.44149402],
       [0.6170564 ],
       [0.5334355 ],
       [0.38663054],
       [0.56980675],
       [0.41918647],
       [0.37718433],
       [0.4777925 ]], dtype=float32)

In [754]:
new_labels[:20]

0     0.25
1        1
2     0.25
3     0.25
4        0
5        1
6     0.25
7     0.25
8        1
9     0.25
10    0.75
11       1
12    0.25
13    0.25
14    0.75
15    0.25
16    0.75
17       1
18    0.25
19    0.75
Name: Stance, dtype: object